# Demo Workflow – Tracing & Evaluation Hooks

Extended workflow with agent-level tracing and evaluation artifacts. This version is LangGraph-ready.

## 1. Setup & Tracing Utilities

In [ ]:

import json, time, uuid
from datetime import datetime

SESSION_ID = str(uuid.uuid4())
TRACE_LOG = "logs/agent_traces.log"

def trace(agent, action, input_data, output_data, status="SUCCESS", start_time=None):
    entry = {
        "timestamp": datetime.utcnow().isoformat(),
        "session_id": SESSION_ID,
        "agent_name": agent,
        "action": action,
        "input_summary": str(input_data)[:300],
        "output_summary": str(output_data)[:300],
        "status": status,
        "latency_ms": int((time.time() - start_time) * 1000) if start_time else None
    }
    with open(TRACE_LOG, "a") as f:
        f.write(json.dumps(entry) + "\n")


## 2. Base Agent with Tracing

In [ ]:

class BaseAgent:
    name = "BaseAgent"

    def run(self, context):
        raise NotImplementedError

    def __call__(self, context):
        start = time.time()
        try:
            output = self.run(context)
            trace(self.name, "run", context, output, start_time=start)
            return output
        except Exception as e:
            trace(self.name, "run", context, str(e), status="FAILURE", start_time=start)
            raise


## 3. Example Agents

In [ ]:

class CaseFinderAgent(BaseAgent):
    name = "CaseFinderAgent"

    def run(self, context):
        cases = ["Case A vs State", "Case B vs Union"]
        context.setdefault("_artifacts", {})["case_laws"] = cases
        return cases


class ArgumentBuilderAgent(BaseAgent):
    name = "ArgumentBuilderAgent"

    def run(self, context):
        arguments = f"Arguments based on {context['_artifacts']['case_laws']}"
        context["_artifacts"]["arguments"] = arguments
        return arguments


## 4. Workflow Execution

In [ ]:

context = {}
case_agent = CaseFinderAgent()
arg_agent = ArgumentBuilderAgent()

case_agent(context)
arg_agent(context)

context


## 5. Evaluation Hooks (aligned with tests/)

In [ ]:

def evaluate(context):
    results = {}

    results['min_cases'] = len(context['_artifacts'].get('case_laws', [])) >= 1
    results['arguments_present'] = bool(context['_artifacts'].get('arguments'))

    return results

evaluation_results = evaluate(context)
evaluation_results


## 6. LangGraph Migration (Structure Only)

The above agents can be directly registered as LangGraph nodes without logic changes.